In [ ]:
import os
import torch
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

In [ ]:
images_dir = "raw"
results_dir = "output"

if not os.path.exists(images_dir):
  os.makedirs(images_dir)

if not os.path.exists(results_dir):
  os.makedirs(results_dir)

if not os.path.exists("masks"):
  os.makedirs("masks")


In [ ]:
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamPredictor

In [ ]:
sam_model = "h"

In [ ]:
if sam_model =="h":
  sam_checkpoint = "sam_vit_h_4b8939.pth"
  model_type = "vit_h"
else:
  sam_checkpoint = "sam_vit_l_0b3195.pth"
  model_type = "vit_l"

In [ ]:
!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

In [ ]:
device = "cuda"

In [ ]:
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

predictor = SamPredictor(sam)

In [ ]:
images = [str("NAO" + str(i) + ".jpeg") for i in range(1, 38)] #37


In [ ]:
!pip install mediapipe

In [ ]:
import mediapipe as mp

In [ ]:
# Calling the pose solution from MediaPipe
mp_pose = mp.solutions.pose

# Calling the solution for image drawing from MediaPipe
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [ ]:
with mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
    for file in images:
        print(file)
        image = cv2.imread(os.path.join(images_dir, file))
        results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        x_min = 10000
        x_max = 0
        y_min = 10000
        y_max = 0
        margin_x = 290
        margin_y = 240
        if not results.pose_landmarks:
            continue
        for landmark in results.pose_landmarks.landmark:
            x_min = min(x_min, landmark.x)
            x_max = max(x_max, landmark.x)
            y_min = min(y_min, landmark.y)
            y_max = max(y_max, landmark.y)

        x_min = int(x_min * image.shape[1])
        x_max = int(x_max * image.shape[1])
        y_min = int(y_min * image.shape[0])
        y_max = int(y_max * image.shape[0])

        x_min = max(0,x_min - margin_x)
        y_min = max(0,y_min - margin_y)
        x_max = min(image.shape[1],x_max + margin_x)
        y_max = min(image.shape[0],y_max + margin_y)

        bbox = np.array([x_min, y_min, x_max, y_max])
        xywh = np.array([x_min, y_min, x_max - x_min, y_max - y_min])

        predictor.set_image(image)

        mask, _, _ = predictor.predict(
          point_coords=None,
          point_labels=None,
          box=bbox,
          multimask_output=False,
        )

        mask = np.array(mask)
        mask = np.where(mask, 1, 0)
        cv2.imwrite(os.path.join("masks", file), mask[0] * 255)

        mask = np.where(mask, 1, 0)

        mask = mask[0] * 255
        mask = mask.astype(np.uint8)
        masked = cv2.bitwise_and(image, image, mask=mask)



        cv2.imwrite(os.path.join(results_dir, file), masked)

In [ ]:
with mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
    for file in images:
        print(file)
        image = cv2.imread(os.path.join(images_dir, file))
        results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        x_min = 10000
        x_max = 0
        y_min = 10000
        y_max = 0
        margin_x = 240
        margin_y = 190
        if not results.pose_landmarks:
            continue
        for landmark in results.pose_landmarks.landmark:
            x_min = min(x_min, landmark.x)
            x_max = max(x_max, landmark.x)
            y_min = min(y_min, landmark.y)
            y_max = max(y_max, landmark.y)

        x_min = int(x_min * image.shape[1])
        x_max = int(x_max * image.shape[1])
        y_min = int(y_min * image.shape[0])
        y_max = int(y_max * image.shape[0])

        x_min = max(0,x_min - margin_x)
        y_min = max(0,y_min - margin_y)
        x_max = min(image.shape[1],x_max + margin_x)
        y_max = min(image.shape[0],y_max + margin_y)

        rs = results.pose_landmarks.landmark[12]
        rs = rs.x*image.shape[1]
        print(rs)

        bbox = np.array([x_min, y_min, x_max, y_max])
        xywh = np.array([x_min, y_min, x_max - x_min, y_max - y_min])


        predictor.set_image(image)

        mask, _, _ = predictor.predict(
          point_coords=None,
          point_labels=None,
          box=bbox,
          multimask_output=False,
        )

        mask = np.array(mask)
        mask = np.where(mask, 1, 0)
        # mask = np.expand_dims(mask, axis=0)
        # mask = mask.astype(np.uint8)
        cv2.imwrite(os.path.join("masks", file), mask[0] * 255)

        contours, _ = cv2.findContours(mask[0].astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # Your call to find the contours
        # threshold input image using otsu thresholding as mask and refine with morphology
        ret, pngmask = cv2.threshold(mask[0].astype(np.uint8), 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        kernel = np.ones((9,9), np.uint8)
        pngmask = cv2.morphologyEx(pngmask, cv2.MORPH_CLOSE, kernel)
        pngmask = cv2.morphologyEx(pngmask, cv2.MORPH_OPEN, kernel)

        result = image.copy()
        result.flags.writeable = True
        result = cv2.rectangle(result, (x_min,y_min), (x_max,y_max), (255, 0, 0), 2)

        # contours, _ = cv2.findContours(mask[0].astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        # contours = sorted(contours, key=cv2.contourArea, reverse=True)
        # print(contours[0])
        mp_drawing.draw_landmarks(
            result,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

        result = cv2.cvtColor(result, cv2.COLOR_BGR2BGRA)
        result[:, :, 3] = pngmask


        cv2.imwrite(os.path.join(results_dir, file), result)

In [ ]:
def get_contour_centroid(contour):
        """Get the centroid of a contour."""
        M = cv2.moments(contour)
        if M['m00'] != 0:
            vertical_coordinate = int(M['m01'] / M['m00'])
            horizontal_coordinate = int(M['m10'] / M['m00'])
        else:
            # if the contour has an area of 0, the centroid cannot be computed this way
            # we use the mean of the contour points instead
            vertical_coordinate, horizontal_coordinate = np.mean(contour, axis=0)[0]
        return int(vertical_coordinate), int(horizontal_coordinate)

def get_largest_contour(image):
        """Get the largest contour in an image."""
        image = cv2.imread(os.path.join(images_dir, image))
        copy = image.copy()

        #laplacian = cv2.Laplacian(image, cv2.CV_8U, ksize=3)
        blur = cv2.GaussianBlur(image, (0, 0), 2)
        gray = cv2.cvtColor(blur, cv2.COLOR_BGR2GRAY)



        ret, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)
        closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_RECT, (15, 15)))

        contours, _ = cv2.findContours(closing, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contours = sorted(contours, key=cv2.contourArea, reverse=True)

        largest_contour = contours[0]
        cv2.drawContours(copy,largest_contour,-1,(0,0,255),4,cv2.LINE_AA)

        vtc, hzt = get_contour_centroid(largest_contour)
        copy = cv2.line(copy, (vtc-50,hzt), (vtc+50,hzt), color=(250, 250, 0), thickness=3)
        copy = cv2.line(copy, (vtc,hzt-50), (vtc,hzt+50), color=(250, 250, 0), thickness=3)
        cv2.imwrite(os.path.join("tests", file), copy)




get_largest_contour(images[6])